# 0.0. Imports

In [1]:
import pickle
import inflection

import numpy as np
import pandas as pd
import seaborn as sn

from xgboost import XGBClassifier
from ydata_profiling import ProfileReport

from sklearn import metrics as mt
from sklearn import ensemble as en
from sklearn import model_selection as ms
from sklearn import preprocessing   as pp

from sklearn import linear_model as lm
from sklearn import ensemble as en


/home/felipe/repos/hackday/hackday/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1.0. Data Description

## 1.1. Loading Data

In [2]:
df_raw = pd.read_csv('data/train.csv')
df_country = pd.read_csv('data/country_base_2.csv')

In [3]:
df1 = pd.merge(df_raw, df_country, on='Nacionalidade', how='left')

In [4]:
df1 = df1.drop('country', axis=1)

## 1.2. Rename Columns

In [5]:
cols_old = ['id', 'Classificacao_do_hotel', 'Meses_da_reserva_ate_o_check-in', 'Numero_de_pernoites_reservadas', 
            'Numero_de_hospedes', 'Regime_de_alimentacao', 'Nacionalidade', 'Forma_de_Reserva',
            'Ja_se_hospedou_anterioremente', 'Tipo_do_quarto_reservado', 'Reserva_feita_por_agencia_de_turismo', 
            'Reserva_feita_por_empresa', 'Reserva_com_Estacionamento', 'Reserva_com_Observacoes', 
            'Reserva_Cancelada', 'latitude', 'longitude']

snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))
df1.columns = cols_new

## 1.3. Data Dimensions

In [6]:
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of cols: {}'.format(df1.shape[1]))

Number of rows: 72159
Number of cols: 17


## 1.4. Check NA

In [7]:
df1.isna().sum()#/len(df1)

id                                         0
classificacao_do_hotel                     0
meses_da_reserva_ate_o_check_in            0
numero_de_pernoites_reservadas             0
numero_de_hospedes                         3
regime_de_alimentacao                      0
nacionalidade                           1093
forma_de_reserva                           0
ja_se_hospedou_anterioremente              0
tipo_do_quarto_reservado                   0
reserva_feita_por_agencia_de_turismo       0
reserva_feita_por_empresa                  0
reserva_com_estacionamento                 0
reserva_com_observacoes                    0
reserva_cancelada                          0
latitude                                1093
longitude                               1093
dtype: int64

### 1.4.1. Replace NA

In [8]:
#df1['numero_de_hospedes'] = df1['numero_de_hospedes'].fillna(2)
df1 = df1.dropna()

In [9]:
# import pandas as pd
# from sklearn.linear_model import LinearRegression
# from sklearn.impute import SimpleImputer

# # carregar dados de exemplo com valores ausentes
# df = pd.read_csv('exemplo.csv')

# # separar os dados em variáveis independentes e dependentes
# X = df.drop('Target', axis=1)
# y = df['Target']

# # criar um objeto SimpleImputer para imputar valores ausentes
# imputer = SimpleImputer(strategy='mean')

# # imputar os valores ausentes usando regressão linear
# for column in X.columns:
#     X_column = X[[column]]
#     X_column_with_nulls = X_column[X_column[column].isnull()]
#     X_column_without_nulls = X_column[X_column[column].notnull()]
#     y_without_nulls = y[X_column[column].notnull()]
#     model = LinearRegression()
#     model.fit(X_column_without_nulls, y_without_nulls)
#     y_pred = model.predict(X_column_with_nulls)
#     X_column_with_nulls[column] = y_pred
#     X[[column]] = pd.concat([X_column_with_nulls, X_column_without_nulls], axis=0)

# # juntar as variáveis independentes e dependentes de volta em um dataframe
# df_imputed = pd.concat([X, y], axis=1)

# # exibir o dataframe resultante
# print(df_imputed)

## 1.5. Data Dtypes

In [10]:
df1.dtypes

id                                        int64
classificacao_do_hotel                   object
meses_da_reserva_ate_o_check_in           int64
numero_de_pernoites_reservadas            int64
numero_de_hospedes                      float64
regime_de_alimentacao                    object
nacionalidade                            object
forma_de_reserva                         object
ja_se_hospedou_anterioremente            object
tipo_do_quarto_reservado                 object
reserva_feita_por_agencia_de_turismo     object
reserva_feita_por_empresa                object
reserva_com_estacionamento               object
reserva_com_observacoes                  object
reserva_cancelada                         int64
latitude                                float64
longitude                               float64
dtype: object

### 1.5.1. Change Dtypes

In [11]:
df1['numero_de_hospedes'] = df1['numero_de_hospedes'].astype('int')

## 1.6. Data Balancing

In [12]:
df1['reserva_cancelada'].value_counts(normalize=True)

0    0.62674
1    0.37326
Name: reserva_cancelada, dtype: float64

## 1.7. Pandas Profiling

In [13]:
# prof = ProfileReport(df_raw)
# prof.to_file(output_file='output.html')

# 2.0. Feature Engineering

In [14]:
df2 = df1.copy()

# 3.0. Data Filtering

In [15]:
df3 = df2.copy()

In [16]:
# reserva_feita_por_agencia_de_turismo reserva_feita_por_empresa
df3 = df3.drop(['reserva_feita_por_agencia_de_turismo', 'reserva_feita_por_empresa'], axis=1)

# 4.0. Data Preparation 

In [17]:
#df4 = df3.drop('nacionalidade', axis=1).copy()
df4 = df3.copy()

In [18]:
#tipo_do_quarto_reservado
# criar um dicionário de frequência de cada categoria
freq_dict = df4['tipo_do_quarto_reservado'].value_counts(normalize=True).to_dict()
# aplicar o encoder ao dataframe
df4['tipo_do_quarto_reservado_encoded'] = df4['tipo_do_quarto_reservado'].map(freq_dict)

In [19]:
df4.head()

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check_in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,reserva_com_estacionamento,reserva_com_observacoes,reserva_cancelada,latitude,longitude,tipo_do_quarto_reservado_encoded
0,33571,5 estrelas,5,5,2,Café da manha e jantar,France,Agência,Não,Amethyst,Sim,Nenhuma,0,46227638.0,2213749.0,0.718447
1,82458,4 estrelas,167,3,2,Café da manha,Spain,Agência,Não,Amethyst,Não,Nenhuma,1,40463667.0,-374922.0,0.718447
2,94061,4 estrelas,4,3,2,Café da manha,Belgium,Agência,Não,Amethyst,Não,1 a 3,0,50503887.0,4469936.0,0.718447
3,75196,4 estrelas,13,2,2,Café da manha,Spain,Agência,Não,Amethyst,Não,Nenhuma,1,40463667.0,-374922.0,0.718447
4,82940,4 estrelas,4,2,2,Café da manha,Spain,Agência,Não,Amethyst,Não,Nenhuma,1,40463667.0,-374922.0,0.718447


In [20]:
# dummy variables
df4_dummy = pd.get_dummies(df4.drop(['id', 'meses_da_reserva_ate_o_check_in', 'numero_de_pernoites_reservadas',
                                    'numero_de_hospedes', 'reserva_cancelada', 'tipo_do_quarto_reservado_encoded',
                                    'latitude', 'longitude', 'tipo_do_quarto_reservado', 'nacionalidade'], axis=1))

df4 = pd.concat([df4[['id', 'meses_da_reserva_ate_o_check_in', 'numero_de_pernoites_reservadas',
                    'numero_de_hospedes', 'reserva_cancelada', 'tipo_do_quarto_reservado_encoded',
                    'latitude', 'longitude']], df4_dummy], axis=1)

# # numerical
# col_num = df4.select_dtypes(include=['int64', 'float64']).columns.tolist()

# # categorical
# col_cat = df4.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]']).columns.tolist()

# # encoding
# df4_dummy = pd.DataFrame(ohe.fit_transform(df4[col_cat]).toarray(), index=df4.index)
# pickle.dump(ohe, open('ohe.pkl', 'wb' ))

# # join numerical and categorical
# df42 = pd.concat([df4[col_num], df4_dummy], axis=1)
# df42.columns = df42.columns.astype(str)

# # one hot encoding
# ohe_class_hotel = pp.OneHotEncoder()
# ohe_aliment = pp.OneHotEncoder()
# ohe_form_reserv = pp.OneHotEncoder()
# ohe_hosp_anter = pp.OneHotEncoder()
# ohe_reserv_estac = pp.OneHotEncoder()
# ohe_reserva_observ = pp.OneHotEncoder()

# # classificacao_do_hotel
# ohe_class_hotel.fit(df4[['classificacao_do_hotel']])
# pickle.dump(ohe_class_hotel, open('ohe_class_hotel.pkl', 'wb' ))
# encoded = ohe_class_hotel.transform(df4[['classificacao_do_hotel']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_class_hotel.get_feature_names_out(['classificacao_do_hotel']))
# df4 = pd.concat([df4, df_encoded], axis=1)

# # regime_de_alimentacao
# ohe_aliment.fit(df4[['regime_de_alimentacao']])
# pickle.dump(ohe_aliment, open('ohe_aliment.pkl', 'wb' ))
# encoded = ohe_aliment.transform(df4[['regime_de_alimentacao']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_aliment.get_feature_names_out(['regime_de_alimentacao']))
# df4 = pd.concat([df4, df_encoded], axis=1)

# # forma_de_reserva
# ohe_form_reserv.fit(df4[['forma_de_reserva']])
# pickle.dump(ohe_form_reserv, open('ohe_form_reserv.pkl', 'wb' ))
# encoded = ohe_form_reserv.transform(df4[['forma_de_reserva']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_form_reserv.get_feature_names_out(['forma_de_reserva']))
# df4 = pd.concat([df4, df_encoded], axis=1)

# # ja_se_hospedou_anterioremente
# ohe_hosp_anter.fit(df4[['ja_se_hospedou_anterioremente']])
# pickle.dump(ohe_hosp_anter, open('ohe_hosp_anter.pkl', 'wb' ))
# encoded = ohe_hosp_anter.transform(df4[['ja_se_hospedou_anterioremente']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_hosp_anter.get_feature_names_out(['ja_se_hospedou_anterioremente']))
# df4 = pd.concat([df4, df_encoded], axis=1)

# # reserva_com_estacionamento
# ohe_reserv_estac.fit(df4[['reserva_com_estacionamento']])
# pickle.dump(ohe_reserv_estac, open('ohe_reserv_estac.pkl', 'wb' ))
# encoded = ohe_reserv_estac.transform(df4[['reserva_com_estacionamento']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_reserv_estac.get_feature_names_out(['reserva_com_estacionamento']))
# df4 = pd.concat([df4, df_encoded], axis=1)

# #reserva_com_observacoes
# ohe_reserva_observ.fit(df4[['reserva_com_observacoes']])
# pickle.dump(ohe_reserva_observ, open('ohe_reserva_observ.pkl', 'wb' ))
# encoded = ohe_reserva_observ.transform(df4[['reserva_com_observacoes']])
# df_encoded = pd.DataFrame(encoded.toarray(), columns=ohe_reserva_observ.get_feature_names_out(['reserva_com_observacoes']))
# df4 = pd.concat([df4, df_encoded], axis=1)

In [21]:
# df4 = df4.drop(['tipo_do_quarto_reservado', 'classificacao_do_hotel', 'regime_de_alimentacao', 'forma_de_reserva',
#          'ja_se_hospedou_anterioremente', 'reserva_com_estacionamento', 'reserva_com_observacoes'], axis=1)


# 5.0. Feature Selection 

In [22]:
df5 = df4.copy()

In [23]:
# split dataset into features and target
X = df5.drop(['id', 'reserva_cancelada'], axis=1)
y = df5['reserva_cancelada']

In [24]:
# split dataset into train and test
X_train, X_val, y_train, y_val = ms.train_test_split(X, y, test_size=0.2)

## 1.5. Data Dtypes

# 6.0. Machine Learning Model

## 6.1. XGBoost

In [25]:
# # model definition
# xgb_model = XGBClassifier()

# # model training
# xgb_model.fit(X_train,y_train.values.ravel())

# # model prediction
# yhat = xgb_model.predict(X_val)

In [26]:
# pickle.dump(xgb_model, open('/home/felipe/repos/hackday/gradient_boosting.pkl', 'wb' ))

## 6.2. Logistic Regression

In [27]:
# # model definition
# lr_model = lm.LogisticRegression()

# # model training
# lr_model.fit(X_train, y_train)

# # model prediction
# yhat = lr_model.predict(X_val)


## 6.3. Gradiente Boosting

In [28]:
# ## model definition
# gdr_model = en.GradientBoostingClassifier(n_estimators=500, random_state=42)

# # model training
# gdr_model.fit(X_train, y_train)

# # model prediction
# yhat = gdr_model.predict(X_val)

## 6.4. Random Forest

In [29]:
# model definition
rf_model = en.RandomForestClassifier(n_estimators=500, random_state=42, n_jobs=-1)

# model training
rf_model.fit(X_train, y_train)

# model prediction
yhat = rf_model.predict(X_val)

In [30]:
pickle.dump(rf_model, open('/home/felipe/repos/hackday/rf_model.pkl', 'wb' ))

# 7.0. Performance

In [31]:
precision = mt.precision_score(y_val, yhat)
recall = mt.recall_score(y_val, yhat)
f1_score = mt.f1_score(y_val, yhat, average='macro')

In [32]:
print('Precision Score: {:.5f}'.format(precision))
print('Precision Recall: {:.5f}'.format(recall))
print('Precision F1 Score: {:.5f}'.format(f1_score))

Precision Score: 0.75082
Precision Recall: 0.68691
Precision F1 Score: 0.77949


# 8.0. Model

In [33]:
df2[df2['nacionalidade']=='Spain']

,id,classificacao_do_hotel,meses_da_reserva_ate_o_check_in,numero_de_pernoites_reservadas,numero_de_hospedes,regime_de_alimentacao,nacionalidade,forma_de_reserva,ja_se_hospedou_anterioremente,tipo_do_quarto_reservado,reserva_feita_por_agencia_de_turismo,reserva_feita_por_empresa,reserva_com_estacionamento,reserva_com_observacoes,reserva_cancelada,latitude,longitude
1,82458,4 estrelas,167,3,2,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1,40463667.0,-374922.0
3,75196,4 estrelas,13,2,2,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1,40463667.0,-374922.0
4,82940,4 estrelas,4,2,2,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1,40463667.0,-374922.0
7,85783,4 estrelas,2,2,1,Café da manha,Spain,B2B,Não,Pink Sapphire,Não,Sim,Não,1 a 3,0,40463667.0,-374922.0
10,109280,4 estrelas,4,5,2,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,0,40463667.0,-374922.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72149,84974,4 estrelas,1,1,1,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,1 a 3,0,40463667.0,-374922.0
72151,56825,4 estrelas,10,7,2,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1,40463667.0,-374922.0
72156,4101,5 estrelas,5,2,2,Café da manha e jantar,Spain,Agência,Não,Amethyst,Não,Não,Não,Nenhuma,1,40463667.0,-374922.0
72157,15828,5 estrelas,1,8,2,Café da manha e jantar,Spain,Balcão,Não,Pink Sapphire,Sim,Não,Não,Nenhuma,0,40463667.0,-374922.0
